In [ ]:
!python yolov5/train.py --img 640 --cfg yolov5/models/yolov5s.yaml --hyp yolov5/data/hyps/hyp.scratch.yaml --batch 8 --epochs 5 --data beer_coke.yaml --weights yolov5/yolov5s.pt --workers 4 --name beer_coke_detector

^C


In [ ]:
!python yolov5\detect.py --source 0 --weights models\best.pt

In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

class CentroidTracker():
	def __init__(self, maxDisappeared=10):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()

		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared

	def register(self, centroid):
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
		self.nextObjectID += 1

	def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		del self.objects[objectID]
		del self.disappeared[objectID]

	def update(self, rects):
		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			for objectID in list(self.disappeared.keys()):
				self.disappeared[objectID] += 1

				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					self.deregister(objectID)

			# return early as there are no centroids or tracking info
			# to update
			return self.objects

		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")

		# loop over the bounding box rectangles
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			# use the bounding box coordinates to derive the centroid
			cX = int((startX + endX) / 2.0)
			cY = int((startY + endY) / 2.0)
			inputCentroids[i] = (cX, cY)

		# if we are currently not tracking any objects take the input
		# centroids and register each of them
		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i])

		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())

			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)

			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value as at the *front* of the index
			# list
			rows = D.min(axis=1).argsort()

			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			cols = D.argmin(axis=1)[rows]

			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()

			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				# val
				if row in usedRows or col in usedCols:
					continue

				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]
				self.objects[objectID] = inputCentroids[col]
				self.disappeared[objectID] = 0

				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)

			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)

			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1

					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)

			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col])

		# return the set of trackable objects
		return self.objects

In [ ]:
model = torch.hub.load('yolov5', 'custom', path='models/yolo_model3.pt', source='local')

# Images
dir = 'yolo_data/images/test/frame_001599.jpg'
imgs = [dir]  # batch of images

# Inference
results = model(imgs)
# or .show(), .save()

print(results.xyxy)

In [ ]:
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import torch
from PIL import Image
import xml.etree.ElementTree as ET
#from google.colab.patches import cv2_imshow


class Take_test_image(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(self.root,"images_test1"))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images_test", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        target={}
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target
        
    def __len__(self):
        return len(self.imgs)


img_array = []
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

ct_beer = CentroidTracker()
ct_coke = CentroidTracker()
#picture = Take_test_image(drive_path, get_transform(train=False))
num_images=10000
model.to(device)
model.eval()
global_label=[]
cap = cv2.VideoCapture("video_vero.mp4")
ret=1
i =0
while i in range(ret+1):

  #path=os.path.join(drive_path, 'images_test1', 'frame_'+str(i).zfill(6)+'.jpg')
  ret, frame = cap.read()

  frame1 = frame
  cv2.imwrite('image.jpg', frame) 
  tic = time.perf_counter()
  torch.cuda.synchronize()
  detection = model("image.jpg")
  detection=detection.xyxy
  #cv2.imshow("image",frame)
  #print(detection)

  box1=detection[0][:,0:4]
 # print(box1)
  scores = detection[0][:,4]
 # print(scores)
  labels=detection[0][:,5]
 # print(labels)
  labels=list(labels)
  scores=list(scores)
  labels1=labels
  box1=list(box1)
  rects_beer=[]
  rects_coke=[]
  m=0
  centerx=[]
	# loop over the detections
  for j in range(len(scores)):
    # filter out weak detections by ensuring the predicted
    # probability is greater than a minimum threshold
    print(scores[j])
    if scores[j] > 0.78:
      # compute the (x, y)-coordinates of the bounding box for
			# the object, then update the bounding box rectangles list
      
      box = np.array(box1[j].cpu())     
			# draw a bounding box surrounding the object so we can
			# visualize it
      (startX, startY, endX, endY) = box.astype("int")
      if labels[j]==0:
        rects_beer.append(box.astype("int"))
        cv2.rectangle(frame1, (startX, startY), (endX, endY),(0, 255, 0), 2)
      if labels[j]==1:
        rects_coke.append(box.astype("int"))
        cv2.rectangle(frame1, (startX, startY), (endX, endY),(0, 0, 255), 2)
	# update our centroid tracker using the computed set of bounding
	# box rectangles
  objects_beer = ct_beer.update(rects_beer)
  objects_coke = ct_coke.update(rects_coke)
	# loop over the tracked objects
  for (objectID, centroid) in objects_beer.items():    
   	# draw both the ID of the object and the centroid of the
		# object on the output frame
    text = "Beer {}".format(objectID)
    cv2.putText(frame1, text, (centroid[0] - 10, centroid[1] - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.circle(frame1, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
  for (objectID, centroid) in objects_coke.items():    
   	# draw both the ID of the object and the centroid of the
		# object on the output frame
    text = "Coke {}".format(objectID)
    cv2.putText(frame1, text, (centroid[0] - 10, centroid[1] - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.circle(frame1, (centroid[0], centroid[1]), 4, (0, 0, 255), -1)
  toc = time.perf_counter()
  torch.cuda.synchronize()
  elapsed_fp = toc-tic
  print(elapsed_fp)



	# show the output frame
  #clear_output()
  cv2.imshow("image",frame1)
  key = cv2.waitKey(1) & 0xFF
  img_array.append(frame1)

	# if the `q` key was pressed, break from the loop
  if key == ord("q"):
   break
height, width, layers = frame.shape
size = (width,height)
out = cv2.VideoWriter('video_traking2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
i=0
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()